In [1]:
import gpu_imports
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cell_images_path = './cell_images'
parasitized_path = cell_images_path + '/Parasitized'
uninfected_path =  cell_images_path + '/Uninfected'

In [3]:
parasitized_folder = os.listdir(parasitized_path)
uninfected_folder = os.listdir(uninfected_path)

In [ ]:
X = []
y = []
dim = (224,224)
count = 0
for image in parasitized_folder:
    try:
        image = cv2.imread(parasitized_path + os.sep + image, cv2.IMREAD_COLOR)
        image = cv2.resize(image, dim)
        X.append(image)
        y.append('Infected')
    except:
        continue

for image in uninfected_folder:
    try:
        image = cv2.imread(uninfected_path + os.sep + image, cv2.IMREAD_COLOR)
        image = cv2.resize(image, dim)
        X.append(image)
        y.append('Uninfected')
    except:
        continue

In [ ]:
target=pd.Series(y, name='target')
print('Number of infected: {}'.format(target.value_counts()[0]))
print('Number of uninfected: {}'.format(target.value_counts()[1]))

figure=plt.figure(figsize=(8,6))
g1=sns.countplot(x=target)
g1.set_xticklabels(['Infected', 'Uninfected']);
g1.set_xlabel('');

In [ ]:
X = np.array(X)
y = np.array(y)

encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(np.int8),
                                                    test_size = 0.2, random_state = 42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
figure, ax = plt.subplots(5,10, figsize = (14,6))
plt.tight_layout()
for row in range(5):
    for col in range(10):
        num = np.random.randint(len(X_train))
        ax[row, col].imshow(X_train[num])
        ax[row, col].tick_params(labelleft=False, labelbottom=False)
        if y[num]==1:
            ax[row, col].set_title('Uninfected')
        else:
            ax[row, col].set_title('Infected')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=40,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip=True,
                                  featurewise_center=True, 
                                  featurewise_std_normalization=True,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2)

train_datagen.fit(X_train)

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_datagen.fit(X_test)

In [ ]:
for x_batch, y_batch in train_datagen.flow(X_train, y_train, batch_size=9):
    for i in range(0,9):
        plt.subplot(330 + 1 + i)
        plt.imshow(x_batch[i], cmap = plt.get_cmap('gray'))
    plt.show()
    break


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dense, Flatten, Dropout, ZeroPadding2D


In [ ]:
model = Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, 
              loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', 
                             verbose=1, save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, 
                      patience=20, verbose=1, 
                      mode='auto')

hist = model.fit_generator(X_train, y_train,                            
                           batch_size=32,
                           epochs=100,
                           validation_split=0.2
                           callbacks=[checkpoint,early])